In [1]:
sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.3 LTS

Matrix products: default
BLAS: /usr/lib/libblas/libblas.so.3.6.0
LAPACK: /usr/lib/lapack/liblapack.so.3.6.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.4.3      R6_2.2.2            magrittr_1.5       
 [4] IRdisplay_0.4.4     pbdZMQ_0.2-6        tools_3.4.3        
 [7] crayon_1.3.4        uuid_0.1-2          stringi_1.1.6      
[10] IRkernel_0.8.6.9000 jsonlite_1.5        stringr_1.2.0      
[13] digest_0.6.12     

In [2]:
library("DESeq2")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min


Attaching packag

In [3]:
setwd("/home/gstupp/projects/Wolan/ana_extractions/analysis/analysis_deseq")

In [4]:
file_path = "X.csv"
countData = read.csv(file_path, row.names = 1)

In [5]:
meta_path = "metadata.csv"
sampleTable = read.csv(meta_path, row.names = 1)
sampleTable = sampleTable[colnames(countData),]
sampleTable$cell = factor(sampleTable$cell)
sampleTable$method = factor(sampleTable$method)
sampleTable$method <- relevel(sampleTable$method, ref="sonic")
sampleTable = sampleTable[sampleTable$cell == "BS",]
sampleTable

,cell,file,method,name.1,pretty_name
BS_pellet_1,BS,data/BS/Pellet/BS pellet 1.txt,pellet,BS_pellet_1,BS pellet 1
BS_pellet_2,BS,data/BS/Pellet/BS pellet 2.txt,pellet,BS_pellet_2,BS pellet 2
BS_pellet_3,BS,data/BS/Pellet/BS pellet 3.txt,pellet,BS_pellet_3,BS pellet 3
BS_sonic_1,BS,data/BS/Sonic/BS sonic 1.txt,sonic,BS_sonic_1,BS sonic 1
BS_sonic_2,BS,data/BS/Sonic/BS sonic 2.txt,sonic,BS_sonic_2,BS sonic 2
BS_sonic_3,BS,data/BS/Sonic/BS sonic 3.txt,sonic,BS_sonic_3,BS sonic 3
BS_triflic_1,BS,data/BS/Triflic/BS triflic 1.txt,triflic,BS_triflic_1,BS triflic 1
BS_triflic_2,BS,data/BS/Triflic/BS triflic 2.txt,triflic,BS_triflic_2,BS triflic 2
BS_triflic_3,BS,data/BS/Triflic/BS triflic 3.txt,triflic,BS_triflic_3,BS triflic 3


## pellet vs sonic

In [6]:
sampleTableX = sampleTable[sampleTable$method %in% c("pellet", "sonic"),]
sampleTableX$method <- relevel(sampleTableX$method, ref="sonic")
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 2, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,BS_pellet_1,BS_pellet_2,BS_pellet_3,BS_sonic_1,BS_sonic_2,BS_sonic_3
C0H3V2,7,6,0,4,7,5
C0H3V3,0,0,0,6,6,9


In [7]:
dds = DESeq(dds)
res = results(dds, alpha = .05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 1343 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 230, 17% 
LFC < 0 (down)   : 281, 21% 
outliers [1]     : 1, 0.074% 
low counts [2]   : 313, 23% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [8]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="BS_deseq_results_pellet_sonic.csv")
resOrdered[0:10,]

log2 fold change (MLE): method pellet vs sonic 
Wald test p-value: method pellet vs sonic 
DataFrame with 10 rows and 6 columns
         baseMean log2FoldChange     lfcSE       stat       pvalue         padj
        <numeric>      <numeric> <numeric>  <numeric>    <numeric>    <numeric>
P0CI73 1016.87614       8.287335 0.5165420  16.043873 6.309428e-58 7.148582e-55
P80239  784.88138      -5.571691 0.4826964 -11.542850 8.022272e-31 4.544617e-28
P26901  286.77461      -4.577485 0.4093435 -11.182505 4.966767e-29 1.875782e-26
P94391  188.81702      -5.561682 0.5128413 -10.844841 2.110014e-27 5.976613e-25
P37877  643.54919       5.014317 0.4834979  10.370918 3.362787e-25 7.620075e-23
P21472  197.54342       6.415380 0.6806517   9.425349 4.286789e-21 8.094887e-19
O31629  147.11204      -7.974936 0.8707598  -9.158594 5.257802e-20 8.510129e-18
P54484  336.97508      11.643915 1.3360740   8.715023 2.906996e-18 4.117033e-16
P40924  622.55961       3.954160 0.4583640   8.626681 6.315827e-18 7.950

## whole (triflic) vs sonic

In [9]:
sampleTableX = sampleTable[sampleTable$method %in% c("triflic", "sonic"),]
sampleTableX$method <- relevel(sampleTableX$method, ref="sonic")
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 2, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,BS_sonic_1,BS_sonic_2,BS_sonic_3,BS_triflic_1,BS_triflic_2,BS_triflic_3
C0H3V2,4,7,5,0,3,4
C0H3V3,6,6,9,0,0,0


In [10]:
dds = DESeq(dds)
res = results(dds, alpha = .05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 1372 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 218, 16% 
LFC < 0 (down)   : 184, 13% 
outliers [1]     : 0, 0% 
low counts [2]   : 372, 27% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [11]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="BS_deseq_results_triflic_sonic.csv")
resOrdered[0:10,]

log2 fold change (MLE): method triflic vs sonic 
Wald test p-value: method triflic vs sonic 
DataFrame with 10 rows and 6 columns
        baseMean log2FoldChange     lfcSE       stat       pvalue         padj
       <numeric>      <numeric> <numeric>  <numeric>    <numeric>    <numeric>
P80239 747.56812      -4.366607 0.2995649 -14.576497 3.963404e-48 4.383525e-45
P54375 329.40898       5.712140 0.4374335  13.058305 5.698257e-39 3.151136e-36
P0CI73 174.45328       5.838238 0.4573601  12.765081 2.568842e-37 9.470464e-35
P26901 280.45142      -3.304884 0.2700305 -12.238929 1.925572e-34 5.324205e-32
P94391 180.93869      -4.243002 0.3568728 -11.889396 1.343752e-32 2.972380e-30
O31629 142.02418      -4.250859 0.3890345 -10.926688 8.592760e-28 1.583932e-25
P71011 137.71276       3.392503 0.3266211  10.386665 2.851509e-25 4.505385e-23
P36945  74.15080      -4.524877 0.5200450  -8.700934 3.291646e-18 4.550700e-16
P40332  68.16917      -4.822504 0.5731659  -8.413801 3.969325e-17 4.877860e-15
O

## Pellet vs. Whole cell (triflic)

In [12]:
sampleTableX = sampleTable[sampleTable$method %in% c("pellet", "triflic"),]
sampleTableX$method <- relevel(sampleTableX$method, ref="triflic")
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 2, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,BS_pellet_1,BS_pellet_2,BS_pellet_3,BS_triflic_1,BS_triflic_2,BS_triflic_3
C0H3V2,7,6,0,0,3,4
C0H3Y1,0,2,0,2,0,4


In [13]:
dds = DESeq(dds)
res = results(dds, alpha = .05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 1299 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 57, 4.4% 
LFC < 0 (down)   : 50, 3.8% 
outliers [1]     : 1, 0.077% 
low counts [2]   : 479, 37% 
(mean count < 3)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [14]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="BS_deseq_results_pellet_triflic.csv")
resOrdered[0:10,]

log2 fold change (MLE): method pellet vs triflic 
Wald test p-value: method pellet vs triflic 
DataFrame with 10 rows and 6 columns
         baseMean log2FoldChange     lfcSE      stat       pvalue         padj
        <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
P0CI73 1309.28428       2.606562 0.3679103  7.084775 1.392708e-12 1.281291e-09
P37877  760.39567       3.381229 0.4864470  6.950869 3.630433e-12 1.669999e-09
P21472  222.45513       4.766075 0.6925537  6.881885 5.906571e-12 1.811348e-09
P54484  399.66360       3.799919 0.6308814  6.023190 1.710129e-09 3.933296e-07
O32028  190.33441       3.239593 0.6234571  5.196177 2.034286e-07 3.743086e-05
P39646   54.39517      -2.846921 0.5539291 -5.139504 2.754647e-07 4.223792e-05
P39751   92.34581       2.227227 0.4637449  4.802698 1.565418e-06 2.012324e-04
P70965  285.49752       2.338043 0.4890938  4.780356 1.749847e-06 2.012324e-04
P39912  273.86010       2.451613 0.5198214  4.716260 2.402194e-06 2.455576e-04